# 📊 Entraînement des modèles ML pour NutriProfil
Ce notebook regroupe les entraînements et sauvegardes de tous les modèles :
- Régression Lasso (prévision consommation)
- Régression linéaire (charge glycémique)
- RandomForestClassifier (risques : diabète, obésité, MCV, cancer colorectal)

Les `.pkl` seront sauvegardés dans `streamlit/`.


In [ ]:
# 📦 Imports
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error, r2_score
import joblib
from pathlib import Path


## 🔢 1. Modèle de prévision de consommation alimentaire (Lasso)

In [ ]:
# Chargement des données conso
df_conso = pd.read_csv('streamlit/data/conso-menages-2024.csv')
cols = [str(y) for y in range(2010, 2025)]
for col in cols:
    df_conso[col] = df_conso[col].astype(str).str.replace(r'\s+', '', regex=True).str.replace(',', '.').astype(float)

X = df_conso[cols[:-1]]
y = df_conso[cols[-1]]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model_conso = Lasso()
model_conso.fit(X_train, y_train)
print("R²:", r2_score(y_test, model_conso.predict(X_test)))
joblib.dump(model_conso, 'streamlit/model_conso.pkl')

## 🍚 2. Modèle charge glycémique (régression linéaire)

In [ ]:
# Chargement données aliments
df_alim = pd.read_csv('streamlit/data/dataframe_complet_rempli_proteines.csv')
df_alim['Famille_regroupee'] = df_alim['Famille_regroupee'].replace('Fruits et lÃ©gumes', 'Fruits et légumes')
df_alim = pd.concat([df_alim, pd.get_dummies(df_alim['Famille_regroupee'], prefix='Famille', drop_first=True)], axis=1)

X = df_alim.drop(columns=['Aliment', 'CG', 'Famille_regroupee','IG'])
y = df_alim['CG']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model_cg = LinearRegression()
model_cg.fit(X_train, y_train)
print("R² CG:", r2_score(y_test, model_cg.predict(X_test)))
joblib.dump(model_cg, 'streamlit/model_charge_glycémique.pkl')

## 🧠 3. Modèles de risques santé (RandomForestClassifier)

In [ ]:
# Réutilisation df_alim pour entraîner 4 modèles de classification binaire
features = df_alim.drop(columns=['Aliment', 'Famille_regroupee', 'IG', 'CG'])
targets = ['diabete', 'obesite', 'mcv', 'cancercolorectal']

# Simuler des colonnes cibles aléatoires pour l'exemple (remplace par ton vrai DataFrame si dispo)
for t in targets:
    df_alim[t] = np.random.randint(0, 2, size=len(df_alim))

for t in targets:
    X_train, X_test, y_train, y_test = train_test_split(features, df_alim[t], test_size=0.2, random_state=42)
    model = RandomForestClassifier()
    model.fit(X_train, y_train)
    joblib.dump(model, f'streamlit/{t}_risque_randomforest.pkl')
    print(f"✅ {t} — R² approx :", model.score(X_test, y_test))